In [69]:
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from emoDB import EmodbDataset
from torch.nn.utils.rnn import pad_sequence


In [88]:
def collate_fn(batch):
    waveforms = [item['waveform'] for item in batch]
    sample_rates = [item['sample_rate'] for item in batch]
    emotions = [item['emotion'] for item in batch]

    # Find max length in waveforms
    max_length = max([waveform.size(1) for waveform in waveforms])

    # Pad all waveforms to max_length
    waveforms_padded = []
    for waveform in waveforms:
        pad_amount = max_length - waveform.size(1)
        waveform_padded = torch.nn.functional.pad(waveform, (0, pad_amount))
        waveforms_padded.append(waveform_padded)

    # Stack everything up
    waveforms_padded = torch.stack(waveforms_padded)
    sample_rates = torch.stack([torch.tensor(sr) for sr in sample_rates])
    emotions = torch.stack([torch.tensor(em) for em in emotions])

    # Create attention mask
    attention_masks_padded = torch.where(waveforms_padded != 0, 1, 0)

    return waveforms_padded, sample_rates, emotions, attention_masks_padded

In [89]:
emoDB = EmodbDataset('./')

c:\UCLA\cs260\realproject\emoDB.py:35: FutureWarning: Could not cast to float32, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  self.df = pd.DataFrame(data, columns=['speaker_id', 'code', 'emotion', 'version', 'file'], dtype=np.float32)


In [95]:
from torch.utils.data import random_split

# Let's say you want to use 80% of the data for training, and 20% for testing
train_size = int(0.8 * len(emoDB))
test_size = len(emoDB) - train_size

train_dataset, test_dataset = random_split(emoDB, [train_size, test_size])

BATCH_SIZE = 4
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [94]:
len(train_dataset)

428